In [1]:
import numpy as np
from scipy import integrate
from scipy.misc import derivative
import warnings

In [2]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [87]:
def linear_combination_function(coefficients, basis_functions):
    def combined_function(x):
        return sum(c * f(x) for c, f in zip(coefficients, basis_functions))
    return combined_function

In [3]:
def trapezoidal_rule(func, x_inicial, x_final, num_points):
    x = np.linspace(x_inicial, x_final, num_points)
    y = func(x)
    h = (x_final - x_inicial) / (num_points - 1)
    integral = (h / 2) * (y[0] + 2 * np.sum(y[1:-1]) + y[-1])
    return integral

In [4]:
def gaussian_quad(func, x_inicial, x_final, args=None):
    def_integral = integrate.quad(func, x_inicial, x_final)[0]
    return def_integral

In [55]:
def galerkin_method(basis_functions, fonte, funcionais, x_inicial, x_final, integration_method, args=None):
    #args = num_partitions caso integration_method == trapezoidal_rule]
    n = len(basis_functions)
    A = np.zeros((n, n))
    l = np.zeros(n) 

    funcional_incognita = funcionais['funcional_incognita']    
    funcional_fonte = funcionais['funcional_fonte']
    
    for i in range(n):
        for j in range(n):
            A[i, j] = funcional_incognita(basis_functions[i], basis_functions[j], integration_method, x_inicial, x_final, args)
            
        l[i] = funcional_fonte(basis_functions[i], fonte, integration_method, x_inicial, x_final, args)

    coef = np.linalg.solve(A, l)
    return coef

## Poisson: $$-u^{"}(x) = f(x), \quad \text{com} \quad u(x_{inicial})=u(x_{final})=0.$$

In [35]:
def funcional_incognita_Poisson(func1, func2, integration_method, x_inicial, x_final, args):
    integrand = lambda x: derivative(func1, x, dx=1e-4) * derivative(func2, x, dx=1e-4)
    return integration_method(integrand, x_inicial, x_final, args)

def funcional_fonte_Poisson(func, fonte, integration_method, x_inicial, x_final, args):
    integrand = lambda x: func(x) * fonte(x)
    return integration_method(integrand, x_inicial, x_final, args)

funcionais_Poisson = {'funcional_incognita': funcional_incognita_Poisson, 'funcional_fonte': funcional_fonte_Poisson}

In [59]:
basis_func_1 = lambda x: np.sin(np.pi * x) / (np.pi)**2 

basis_func_2 = lambda x: x*(x-1)

basis_func_3 = lambda x: x*(x-1)**2

basis_func_4 = lambda x: (x**2)*(x-1)

f = lambda x: np.sin(np.pi * x)

basis_functions = [basis_func_1, basis_func_2, basis_func_3]
x_inicial = 0
x_final = 1
num_partitions = 100

coef_gaussian = galerkin_method(basis_functions, f, funcionais_Poisson, x_inicial,
                                x_final, gaussian_quad)
coef_gaussian

array([ 1.00000115e+00,  4.40088816e-07, -1.09137987e-13])

In [82]:
solucao = linear_combination_function(coef_gaussian, basis_functions)

In [86]:
solucao(np.linspace(0,1,100))

array([0.00000000e+00, 3.21471108e-03, 6.42618531e-03, 9.63118900e-03,
       1.28264950e-02, 1.60088858e-02, 1.91751571e-02, 2.23221208e-02,
       2.54466080e-02, 2.85454727e-02, 3.16155946e-02, 3.46538823e-02,
       3.76572765e-02, 4.06227531e-02, 4.35473260e-02, 4.64280505e-02,
       4.92620260e-02, 5.20463988e-02, 5.47783653e-02, 5.74551746e-02,
       6.00741315e-02, 6.26325989e-02, 6.51280005e-02, 6.75578238e-02,
       6.99196221e-02, 7.22110173e-02, 7.44297021e-02, 7.65734425e-02,
       7.86400799e-02, 8.06275334e-02, 8.25338019e-02, 8.43569658e-02,
       8.60951894e-02, 8.77467224e-02, 8.93099019e-02, 9.07831539e-02,
       9.21649949e-02, 9.34540336e-02, 9.46489720e-02, 9.57486069e-02,
       9.67518310e-02, 9.76576342e-02, 9.84651045e-02, 9.91734287e-02,
       9.97818936e-02, 1.00289887e-01, 1.00696896e-01, 1.01002513e-01,
       1.01206428e-01, 1.01308437e-01, 1.01308437e-01, 1.01206428e-01,
       1.01002513e-01, 1.00696896e-01, 1.00289887e-01, 9.97818936e-02,
      

Preciso criar os funcionais do primeiro problema da Lista.